## OpenSearch Warming Up

In [1]:
from typing import List, Tuple
from opensearchpy import OpenSearch, RequestsHttpConnection

### OpenSearch Client 생성

In [2]:
def create_aws_opensearch_client(region: str, host: str, http_auth: Tuple[str, str]) -> OpenSearch:
  aws_client = OpenSearch(
    hosts=[{'host': host.replace("https://",""), 'port': 443}],
    http_auth=http_auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
  )

  return aws_client

aws_region = "ap-northeast-2"
opensearch_cluster_domain='https://search-bedrock-fkik524xhgkrn3t2omdtfspzmm.ap-northeast-2.es.amazonaws.com'
http_auth=('admin', 'Admin123@')
aws_client = create_aws_opensearch_client(aws_region, opensearch_cluster_domain, http_auth)
aws_client

<OpenSearch([{'host': 'search-bedrock-fkik524xhgkrn3t2omdtfspzmm.ap-northeast-2.es.amazonaws.com', 'port': 443}])>

### Index Creation

In [3]:
def create_index(aws_client, index_name, index_body):
  response = aws_client.indices.create(index_name, index_body)
  print("\nCreating Index: ")
  print(response)

# Create an index with non-default setting
index_name = "python-test-index"
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 2
    }
  }
}
create_index(aws_client, index_name, index_body)


Creating Index: 
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


### Check if index exists

In [4]:
def check_if_index_exist(aws_client, index_name):
  exists = aws_client.indices.exists(index_name)
  print(f"index_name={index_name}, exists={exists}")
  return exists

index_exists = check_if_index_exist(aws_client, index_name)
index_exists

index_name=python-test-index, exists=True


True

### Add Documents

In [6]:
# Add a document to the index
def add_doc(aws_client, index_name, document, id):
  response = aws_client.index(
    index=index_name,
    body=document,
    id=id,
    refresh=True
  )
  print("\nAdding document:")
  print(response)

document = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}
id = 1
add_doc(aws_client, index_name, document, id)

document = {
  'title': 'Parasite',
  'director': 'Junho Bong',
  'year': '2019'
}
id = 2
add_doc(aws_client, index_name, document, id)


Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 2, 'result': 'updated', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}


### Search Document

In [7]:
# search doc
def search_document(aws_client, query, index_name):
  response = aws_client.search(
    body=query,
    index=index_name
  )
  print("\nSearch results:")
  print(response)

q = 'miller'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title', 'director']
    }
  }
}
search_document(aws_client, query, index_name)


Search results:
{'took': 66, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_index': 'python-test-index', '_id': '1', '_score': 0.6931471, '_source': {'title': 'Moneyball', 'director': 'Bennett Miller', 'year': '2011'}}]}}


In [8]:
### delete doc
def delete_doc(aws_client, index_name, id):
  response = aws_client.delete(
    index=index_name,
    id=id
  )
  print("\nDeleting doc")
  print(response)
id=1
delete_doc(aws_client, index_name, id)


Deleting doc
{'_index': 'python-test-index', '_id': '1', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}


In [9]:
# Delete index
def delete_index(aws_client, index_name):
  response = aws_client.indices.delete(
    index=index_name
  )
  print("\nDeleting Index:")
  print(response)

delete_index(aws_client, index_name)


Deleting Index:
{'acknowledged': True}
